# Домашнее задание №2: выделение коллокаций

In [32]:
import re
import nltk
from nltk.collocations import *
from nltk.tokenize import word_tokenize
from nltk.metrics.spearman import *

In [2]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

предобрабтка корпуса

In [12]:
docs = []
court_docs = open('court-V-N.csv', 'r', encoding='utf-8').read().split('\n')
for doc in court_docs:
    doc = re.sub(',', '', doc)
    doc = word_tokenize(doc)
    docs.append(doc)

тепреь найдем коллокации

In [15]:
tr_finder = TrigramCollocationFinder.from_documents(docs)

In [27]:
tr_likl = tr_finder.nbest(trigram_measures.likelihood_ratio, 10)
tr_likl

[('СУД', 'СОГЛАСИТЬСЯ', 'СУД'),
 ('СУД', 'ПЕРЕДАТЬ', 'СУД'),
 ('СУД', 'ПОЙТИ', 'СУД'),
 ('СУД', 'ПРИЗНАТЬ', 'РЕШЕНИЕ'),
 ('СУД', 'УДОВЛЕТВОРИТЬ', 'ИСК'),
 ('СУД', 'ПРИЗНАТЬ', 'ВИНОВНАЯ'),
 ('СУД', 'УДОВЛЕТВОРИТЬ', 'ХОДАТАЙСТВО'),
 ('СУД', 'ПРИНЯТЬ', 'РЕШЕНИЕ'),
 ('СУД', 'ПРИЗНАТЬ', 'ПРАВОТА'),
 ('ВЫДАТЬ', 'САНКЦИЯ', 'АРЕСТ')]

In [28]:
tr_st = tr_finder.nbest(trigram_measures.student_t, 10)
tr_st

[('СУД', 'УДОВЛЕТВОРИТЬ', 'ИСК'),
 ('СУД', 'ПРИНЯТЬ', 'РЕШЕНИЕ'),
 ('СУД', 'УДОВЛЕТВОРИТЬ', 'ХОДАТАЙСТВО'),
 ('СУД', 'ВЫДАТЬ', 'САНКЦИЯ'),
 ('СУД', 'ПРИЗНАТЬ', 'ВИНОВНАЯ'),
 ('СУД', 'ВЫНЕСТИ', 'РЕШЕНИЕ'),
 ('ВЫДАТЬ', 'САНКЦИЯ', 'АРЕСТ'),
 ('СУД', 'НАЛОЖИТЬ', 'АРЕСТ'),
 ('СУД', 'САНКЦИОНИРОВАТЬ', 'АРЕСТ'),
 ('СУД', 'ОТКАЗАТЬ', 'УДОВЛЕТВОРЕНИЕ')]

In [29]:
tr_chi = tr_finder.nbest(trigram_measures.chi_sq, 10)
tr_chi

[('АРМИЯ', 'ВОССТАНОВИТЬСЯ', 'ИНСТИТУТ'),
 ('АССОЦИИРОВАТЬСЬ', 'БОЛЬШОЙ', 'ПОЛИТИКА'),
 ('БЫТНОСТЬ', 'МИНИСТР', 'ПОДЧИНЕННЫЙ'),
 ('ВРАГ', 'НАРОД', 'КОЛЛАЖ'),
 ('ДЕКАБРЬ', 'ПОТОМ', 'ВООБЩЕ'),
 ('ДЕСЯТКА', 'КОРРУМПИРОВАТЬ', 'МИРОВОЙ'),
 ('ДОКЛАД', 'ПАРЛАМЕНТСКИЙ', 'АССАМБЛЕЯ'),
 ('ЗВАНИЕ', 'ЗАСЛУЖЕННЫЙ', 'МАШИНОСТРОИТЕЛЬ'),
 ('ИНВАЛИД', 'ДЕНЬГА', 'УДЕРЖАТЬ'),
 ('КАСЬЯНОВ', 'ДОБРОСОВЕСТНЫЙ', 'ПРИОБРЕТАТЕЛЬ')]

In [30]:
tr_pmi = tr_finder.nbest(trigram_measures.pmi, 10)
tr_pmi

[('АРМИЯ', 'ВОССТАНОВИТЬСЯ', 'ИНСТИТУТ'),
 ('АССОЦИИРОВАТЬСЬ', 'БОЛЬШОЙ', 'ПОЛИТИКА'),
 ('БЫТНОСТЬ', 'МИНИСТР', 'ПОДЧИНЕННЫЙ'),
 ('ВРАГ', 'НАРОД', 'КОЛЛАЖ'),
 ('ДЕКАБРЬ', 'ПОТОМ', 'ВООБЩЕ'),
 ('ДЕСЯТКА', 'КОРРУМПИРОВАТЬ', 'МИРОВОЙ'),
 ('ДОКЛАД', 'ПАРЛАМЕНТСКИЙ', 'АССАМБЛЕЯ'),
 ('ЗВАНИЕ', 'ЗАСЛУЖЕННЫЙ', 'МАШИНОСТРОИТЕЛЬ'),
 ('ИНВАЛИД', 'ДЕНЬГА', 'УДЕРЖАТЬ'),
 ('КАСЬЯНОВ', 'ДОБРОСОВЕСТНЫЙ', 'ПРИОБРЕТАТЕЛЬ')]

видим, что самые лучшие метрики student_t и likelihood_ratio, student_t даже лучше

загружаем золотой стандарт

In [42]:
gold = []
g = open('gold_court.txt', 'r', encoding='utf-8').read().split('\n')
for doc in g:
    doc = re.sub('\ufeff', '', doc)
    doc = doc.split(',')
    gold.append(doc)
print(gold)

[['СУД', 'ПРИЗНАТЬ', 'ВИНОВНАЯ'], ['СУД', 'УДОВЛЕТВОРИТЬ', 'ИСК'], ['СУД', 'ОТКЛОНИТЬ', 'ИСК'], ['СУД', 'ПОДАТЬ', 'ИСК'], ['СУД', 'УДОВЛЕТВОРИТЬ', 'ЖАЛОБА'], ['СУД', 'ОТМЕНИТЬ', 'УКАЗ'], ['СУД', 'ПРОДЛИТЬ', 'СРОК'], ['СУД', 'РАССМОТРЕТЬ', 'АПЕЛЛЯЦИЯ'], ['СУД', 'ВОЗОБНОВИТЬ', 'СЛУШАНИЕ'], ['СУД', 'ПРОВЕСТИ', 'СЛУШАНИЕ']]


теперь я сравниваю золотой стандарт и nbest

In [43]:
def normalize(arr):
    new_arr = [' '.join(line) for line in arr]
    return new_arr

In [55]:
print('%0.1f' % spearman_correlation(ranks_from_sequence(normalize(gold)), ranks_from_sequence(normalize(tr_st))))

-16.0


In [59]:
print('%0.1f' % spearman_correlation(ranks_from_sequence(normalize(gold)), ranks_from_sequence(normalize(tr_likl))))

-33.0


В результате, я посчитала коллокации (триграммы) при помощи всех возможных характеристик и на взгляд мне больше всего понравились student_t и likelihood_ratio. Потом я посчитала ранговый коэффициент корреляции Спирмена между 10 лучшими результатами(student_t/likelihood_ratio) и золотым стандартом. 